# Laboratory 1
    Authors:
    - Roberto Pérez 
    - Arturo Bringas
    - Edgar Bazo
    - Mariana Lugo

# Imports

#### Python libraries

In [ ]:
import sys
import pandas as pd
import numpy as np
import re
import unicodedata
from pandas_profiling import ProfileReport
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import probscale
from scipy import stats

#### Acillary modules

In [ ]:
%reload_ext autoreload
%autoreload 2

from utils.lab1_funcs import *

# Loading data

Data downloaded from -> https://datos.cdmx.gob.mx/explore/dataset/consumo-agua/export/

In [ ]:
df = pd.read_csv("data/consumo-agua.csv")

In [ ]:
df

# Exploratory Data Analysis (EDA)
## Data profiling

## ¿Cuántas variables tenemos?

In [ ]:
count_vars(df)

## ¿Cuántas observaciones tenemos?

In [ ]:
count_obs(df)

## ¿Cuántas observaciones únicas tenemos por variable?

In [ ]:
count_unique_obs(df)

## ¿Cuántas variables numéricas tenemos?

Tenemos 8 variables numéricas

In [ ]:
vars_num = ["consumo_total",
    "consumo_total_dom",
    "consumo_total_no_dom", 
    "consumo_total_mixto",
    "consumo_prom",        
    "consumo_prom_dom",
    "consumo_prom_no_dom",
    "consumo_prom_mixto"]

In [ ]:
count_type_vars(vars_num, "numerica")

## ¿Cuántas variables de fecha tenemos?
- Para efectos de este ejercicio, no hay ninguna variable de tipo fecha, o que consideremos de fecha.

## ¿Cuántas variables categóricas tenemos?

In [ ]:
cat_vars = [
    "anio",
    "nomgeo",
    "alcaldia",
    "colonia",
    "bimestre",
    "indice_des",
]

In [ ]:
count_type_vars(cat_vars, "categórica")

## ¿Cuántas variables de texto tenemos?
- Para efectos de este ejercicio, no hay ninguna variable de tipo texto, o que consideremos de texto. Se podría considerar a la variable `gid` como un identificador de texto.

In [ ]:
vars_text = ["gid"]

In [ ]:
count_type_vars(vars_text, "texto")

## Generea el profiling de cada variable

### Numeric data profiling

- [x] Tipo de dato: float, integer
- [x] Número de observaciones
- [x] Mean
- [x] Desviación estándar
- [x] Cuartiles: 25%, 50%, 75%
- [x] Valor máximo
- [x] Valor mínimo
- [x] Número de observaciones únicos
- [x] Top 5 observaciones repetidas
- [x] Número de observaciones con valores faltantes
- [x] ¿Hay redondeos? -> Se observa que los datos de los totales en el consumo (dom, mixto, no_dom) están a dos decimales desde la fuente. Los datos de los promedios del consumo (dom, mixto, no_dom) tienen seis decimales desde la fuente.  No podemos asegurar que los datos, tanto de los totales, como de los promedios,  están redondeados.

#### Función para perfil de datos numérico

In [ ]:
## Data profiling compacted in function
data_profiling_numeric(df, vars_num)

### Categoric data profiling

**Profiling: Variables categóricas**

- [x] Número de categorías
- [x] Valor de las categorías
- [x] Moda
- [x] Valores faltantes
- [x] Número de observaciones con valores faltantes
- [x] Proporción de observaciones por categoría
- [x] Top 1, top 2, top 3 (moda 1, moda 2, moda 3)
- [x] Faltas de ortografía ?

In [ ]:
#data profiling function
data_profiling_categ(df,cat_vars)

### Additional data profiling

- En este ejercicio particular no se contó con variables del siguiente tipo para hacer el profiling:
    - Imágen
    - Audio

## ¿Qué conocemos ahora de este set de datos por variable?

1. ¿Cuántas alcadías tienes?
    - Hay un total de 16 alcaldías

2. ¿Cuántos `nomgeo` tienes?
    - Hay un total de 17 nomgeo

3. ¿Identificas algún error?
    - Hay una categoría con un error ortográfico: Existen 2140 observaciones en la columna **nomgeo** que dice: Talpan y 1064 observaciones que dicen: Tlalpan

## Transformar el nombre de las columnas a formato estándar: minúsculas, sin espacios en blanco -cambiar por guiónes bajos-, sin signos de puntuación

In [ ]:
df_cleancols = clean_col_names(df)

In [ ]:
df_cleancols

## Transformación de variables geoespaciales
- Agregar la variable `latitud` y `longitud` generadas a partir de la column `geo_point`.
- Pasar la variable `latitud` y `longitud` a numérica -si no la tomó como numérica-.
- Eliminar la columna `geo_point` -una vez que creaste la variable `latitud` y `longitud`.
- Eliminar la columna `geo_shape`.
- Cambiar a minúsculas las columnas `alcaldía`, `colonia` e `indice_des`.

In [ ]:
df_geotransform = geo_transformation(df_cleancols, "geo_point", "geo_shape")
df_geotransform

## Geospatial data profiling

In [ ]:
geo_vars = ["latitud", "longitud"]

geo_vars_precision(df_geotransform, geo_vars)

- Cambiar a minúsculas las columnas `alcaldía`, `colonia` e `indice_des`.

In [ ]:
vars_lower=["indice_des", "alcaldia", "colonia","nomgeo"]

df_lower_values = convert_lower(df_geotransform, vars_lower)

## Corrección de observaciones seleccionadas

- Nótese que hay una entrada llamada "talpan" que debería ser "tlalpan"

In [ ]:
 df_lower_values["nomgeo"].value_counts()

In [ ]:
dicc_cor = {
    "nomgeo": {
        "talpan": "tlalpan"
    }
}

In [ ]:
df_correct = correct_selected_entries(df_lower_values, dicc_cor)

In [ ]:
df_correct["nomgeo"].value_counts() 

# Review Changes

## ¿Cuántas variables tenemos?

In [ ]:
count_vars(df_correct)

## ¿Cuántas observaciones tenemos?

In [ ]:
count_obs(df_correct)

## ¿Cuántas observaciones únicas tenemos por variable?

In [ ]:
count_unique_obs(df_correct)

## ¿Cuántas variables numéricas tenemos?

In [ ]:
count_type_vars(vars_num, "numerica")

## ¿Cuántas variables de fecha tenemos?
- Para efectos de este ejercicio, no hay ninguna variable de tipo fecha, o que consideremos de fecha.

## ¿Cuántas variables categóricas tenemos?

In [ ]:
count_type_vars(cat_vars, "categórica")

## ¿Cuántas variables de texto tenemos?
- Para efectos de este ejercicio, no hay ninguna variable de tipo texto, o que consideremos de texto. Se podría considerar a la variable `gid` como un identificador de texto.

In [ ]:
count_type_vars(vars_text, "texto")

## Genera el profiling de cada variable

In [ ]:
## Data profiling compacted in function
data_profiling_numeric(df_correct, vars_num)

In [ ]:
df_correct["nomgeo"].value_counts() 

In [ ]:
#data profiling function
data_profiling_categ(df_correct,cat_vars)

## Data Profiling con Pandas-Profiling

In [ ]:
profile = ProfileReport(df_correct, title="Pandas Profiling Report", explorative = True)

In [ ]:
profile

In [ ]:
profile.to_file("Profile_variables.html")

# Graphic Exploratory Data Analysis (GEDA)

## Análisis Univariado

### *Variables Categóricas*

#### Barplots
La función que diseñamos para crear gráficas de barras está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
    - "anio"
    - "nomgeo",
    - "alcaldia",
    - "colonia",
    - "bimestre",
    - "indice_des",

In [ ]:
df_plot = df_correct.copy()

In [ ]:
barplot_cat(df_plot, "alcaldia", tops=10)

### *Varibles Númericas*

Se definen las listas de las variables a explorar:

In [ ]:
vars_num_tot= [
    "consumo_total",
    "consumo_total_dom",
    "consumo_total_no_dom",
    "consumo_total_mixto",
    "indice_des"]

vars_num_prom= [
    "consumo_prom",
    "consumo_prom_dom",
    "consumo_prom_no_dom",
    "consumo_prom_mixto",
    "indice_des"]

vars_num_i= [
    "consumo_total",
    "consumo_total_dom",
    "consumo_total_no_dom",
    "consumo_total_mixto",
    "consumo_prom",
    "consumo_prom_dom",
    "consumo_prom_no_dom",
    "consumo_prom_mixto",
    "indice_des",
    "alcaldia",
    "bimestre"]

#### Histogramas

La función que diseñamos para crear histogramas está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
histograms_numeric_total(df_plot, "consumo_total")

A partir de la exploración de los datos numéricos por medio de histogramas, se notó la presencia de muchos datos atípicos.

### Distribución del consumo de agua por índice de desarrollo.

Se muestran los histogramas del consumo (variables numéricas) por cada categoría del índice de desarrollo(`indice_des`). 

Con el objetivo de tener una mejor observación del comportamiento de la distribución del consumo (totales y promedios), se transformaron las variables en escala logarítmica.

Sin embargo, de acuerdo con el data profiling en el EDA, se observó que para todos los consumos (totales y promedio), el valor top 1 es *cero*. Por lo tanto, la distribuciones logarítmicas no mostrarán estos valores.

Con base en lo anterior, y dado que la granularidad de los datos es a nivel manzana, ¿Existen muchas manzanas sin consumo de agua? ¿Es correcto lo anterior o es un error? Contestaremos la pregunta más adelante.

La función que diseñamos para crear histogramas está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
histograms_numeric(df_plot, "consumo_prom_mixto", "indice_des")

#### Boxplots

Los boxplots reafirman que para todas las variables numéricas existen outliers, incluso por cada categoría del índice desarrollo.  

La función que diseñamos para crear gráficas de baja y brazos está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
box_plot_num(df_plot, "indice_des", "consumo_total_mixto")

#### Scatterplots

La función que diseñamos para crear scatterplots está pensada para que el usuario especifique las 2 variables que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
  - "consumo_total_mixto",
  - "consumo_total_dom",
  - "consumo_total_no_dom",
  - "consumo_total"

* Variables totales del consumo: 

In [ ]:
scatterPlotFacet(df_plot, "consumo_total_dom", "consumo_total_no_dom", "indice_des", "bimestre")

* Variables promedio del consumo:

- La lista de variables que se pueden graficar con esta función es:
    - "consumo_prom_dom",
    - "consumo_prom_mixto",
    - "consumo_prom_no_dom",
    - "consumo_prom"

In [ ]:
scatterPlotFacet(df_plot, "consumo_prom_dom", "consumo_prom_no_dom", "indice_des", "bimestre")

#### Rugplot

La función que diseñamos para crear gráficas de tapete está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
rugplot_num(df_plot, "consumo_prom_no_dom")

#### Density Estimate

La función que diseñamos para crear gráficas de densidad está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
distplot_num(df_plot, "consumo_prom_mixto", 85)

#### QQ-Plot

La función que diseñamos para crear gráficas qq está pensada para que el usuario especifique la variable que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
qq_plot(data = df_plot, variable = "consumo_total", ymin = 1, ymax = 200)

#### Matriz de Correlación

In [ ]:
corr_plot(data = df, variables = vars_num, title = "Matriz de correlación")

## Análisis Multivariado

#### Distribución del consumo de agua por categoría del ínidice de desarrollo en cada alcaldía.

* Conteo de registros por alcaldía por `indice_des`

In [ ]:
create_heatmap(df_plot, "alcaldia", "indice_des", "gid")

Con el mapa de calor por alcaldía podemos ver que sí hay ubicaciones específicas con una tendencia clara a ser clasificadas con un indice_des particular (e.g. si el registro es de Azcapotzalco, tenderá a ser clasificado como "bajo"). (el mapa de calor muestra la proporción de conteos de cada clasificación por alcaldía)

#### Histogramas de distribución del consumo por índice de desarrollo humano y variables categóricas.

La función que diseñamos para crear este grupo de histogramas está pensada para que el usuario especifique la variable de consumo que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
histograms_numeric_rv_cat(df_plot, "consumo_total", "indice_des", "alcaldia")

#### Boxplot del consumo por alcaldía:

La función que diseñamos para crear esta visualización de histogramas está pensada para que el usuario especifique la variable de consumo que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
box_plot_num_location(df_plot, "consumo_total_no_dom", "iztapalapa")

#### Distribución del consumo de agua por categoría del ínidice de desarrollo por bimestre.

La función que diseñamos para crear esta visualización de histogramas está pensada para que el usuario especifique la variable de consumo que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
histograms_numeric_rv_cat(df_plot, "consumo_total", "indice_des", "bimestre")

#### Distribución del consumo de agua por categoría del ínidice de desarrollo por colonia.

Se enlistan las colonias top 15 con mayor número de observaciones:

In [ ]:
df_colonia = df_plot[df_plot["colonia"].isin(colonia_top_15)]

La función que diseñamos para crear esta visualización de histogramas está pensada para que el usuario especifique la variable de consumo que desea visualizar.
- La lista de variables que se pueden graficar con esta función es:
     - 'consumo_total',
     - 'consumo_total_dom',
     - 'consumo_total_no_dom',
     - 'consumo_total_mixto',
     - 'consumo_prom',
     - 'consumo_prom_dom',
     - 'consumo_prom_no_dom',
     - 'consumo_prom_mixto'

In [ ]:
histograms_numeric_rv_cat(df_colonia, "consumo_total", "indice_des", "colonia")

#### Distribución espacial de la variable de respuesta `indice_des` 

In [ ]:
scatter_map(df_plot)

#### Consistencia  de la clasificación de la variable `colonia` con el `indice_des`.

Lo que nos interesa averiguar con este analisis es si las distintas colonias están clasificadas con una sola etiqueta de índice de desarrollo (e.g. la colonia "Navidad" siempre es clasificada como "popular")

In [ ]:
colonia_devidx_consistency(df_plot)

Con esta gráfica nos damos cuenta de que no hay mucha consistencia en la clasificación. Solo el 21% de las las colonias fueron consistentemente clasificadas con una sola etiqueta de la variable `indice_des`.

---
---